앙상블 모형 개선4, K-Fold 사용  
mae = 1.0827

## 데이터 전처리

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import seaborn as sns


# 1. 데이터 로드
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")
submission_df = pd.read_csv("data/sample_submission.csv")

# test.csv에서 원본 id 저장
test_ids = test_df["id"].copy()
test_df = test_df.drop(columns=["id"])  # 이후 분석을 위해 삭제

In [ ]:
# 2. 데이터 전처리
# 'Sex' 라벨 인코딩
label_encoder = LabelEncoder()
train_df["Sex"] = label_encoder.fit_transform(train_df["Sex"])
test_df["Sex"] = label_encoder.transform(test_df["Sex"])

# Height가 0인 경우 평균값으로 대체
height_mean = train_df.loc[train_df["Height"] > 0, "Height"].mean()
train_df.loc[train_df["Height"] == 0, "Height"] = height_mean
test_df.loc[test_df["Height"] == 0, "Height"] = height_mean

# IQR 기반 이상치 제거 함수 정의 및 적용
def remove_outliers_iqr(df, cols, threshold=1.5):  # threshold=3.0에서 변경
    Q1 = df[cols].quantile(0.25)
    Q3 = df[cols].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - threshold * IQR
    upper_bound = Q3 + threshold * IQR
    return df[~((df[cols] < lower_bound) | (df[cols] > upper_bound)).any(axis=1)]

num_cols = train_df.select_dtypes(include=["float64"]).columns
train_df = remove_outliers_iqr(train_df, num_cols, threshold=1.5)

# 중복 데이터 제거 및 'id' 컬럼 삭제
train_df = train_df.drop_duplicates().drop(columns=["id"])

# X, y 분리
X = train_df.drop(columns=["Age"])
y = train_df["Age"]

# # Train/Validation Split (8:2 비율)
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. 스케일링
scaler = StandardScaler()  # MinMaxScaler()에서 변경
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
test_scaled = scaler.transform(test_df)  # test data도 스케일링 적용

# 4. PCA 적용 (95% 이상의 분산을 설명하는 주성분 선택)
pca = PCA(n_components=0.95)
X_train_pca = pca.fit_transform(X_train_scaled)
X_valid_pca = pca.transform(X_valid_scaled)
test_pca = pca.transform(test_scaled)  # test data에도 pca 적용

## 모델 학습

In [16]:
# 5. 개별 모델 최적화 (GridSearchCV 적용) - 최적의 하이퍼파라미터
# RandomForestRegressor와 GradientBoostingRegressor에만 적용

# 5-1. RandomForestRegressor 튜닝
rf_param_grid = {
    "n_estimators": [50, 100, 200],
    "max_depth": [5, 10, 20], # 기존보다 낮춰서 과적합 방지
    "min_samples_split": [5, 10],  # 더 큰 값으로 설정해 분할 제한
    "max_features" : [0.5, "sqrt", "log2"]  # 특성 수 제한
}
rf_grid_search = GridSearchCV(RandomForestRegressor(random_state=42), rf_param_grid, 
                              scoring="neg_mean_absolute_error", cv=5, n_jobs=-1)
rf_grid_search.fit(X_train_scaled, y_train)

print(f"Best RandomForest Params: {rf_grid_search.best_params_}")

# 개별 모델 정의, 최적모델 할당
rf_model = RandomForestRegressor(**rf_grid_search.best_params_, random_state=42)

Best RandomForest Params: {'max_depth': 10, 'max_features': 0.5, 'min_samples_split': 10, 'n_estimators': 200}


In [17]:
# 5-2. GradientBoostingRegressor 튜닝
gb_param_grid = {
    "n_estimators": [50, 100, 200],
    "learning_rate": [0.01, 0.05, 0.1],
    "max_depth": [3, 5, 7],
    "subsample" : [0.7, 0.8, 0.9]  # 과적합 방지용
}
gb_grid_search = GridSearchCV(GradientBoostingRegressor(random_state=42), gb_param_grid, 
                              scoring="neg_mean_absolute_error", cv=5, n_jobs=-1)
gb_grid_search.fit(X_train_scaled, y_train)

print(f"Best GradientBoosting Params: {gb_grid_search.best_params_}")

# 개별 모델 정의, 최적모델 할당
gb_model = GradientBoostingRegressor(**gb_grid_search.best_params_, random_state=42)

Best GradientBoosting Params: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200, 'subsample': 0.8}


In [18]:
# 6. 앙상블 모델 (Voting Regressor) 설정
# 개별 모델 정의
# RandomForest와 GradientBoosting은 위에서 튜닝하면서 정의함.
lr_model = LinearRegression()  # 선형회귀

# Voting Regressor 설정 (모델들을 평균으로 결합)
ensemble_model = VotingRegressor(estimators=[
    ('lr', lr_model),
    ('rf', rf_model),
    ('gb', gb_model)
], weights=[0.5, 1, 2])  # 가중치 조정, GradientBoostingRegressor에 더 높은 가중치 부여

# 7. 모델 학습
ensemble_model.fit(X_train_scaled, y_train)

# 8. 검증 데이터 예측
y_pred = ensemble_model.predict(X_valid_scaled)
mae = mean_absolute_error(y_valid, y_pred)
print(f"Validation MAE with Voting Regressor: {mae:.4f}")

Validation MAE with Voting Regressor: 1.0827


In [19]:
# 9. 테스트 데이터 예측
test_preds = ensemble_model.predict(test_scaled)

# 10. 제출 파일 생성 (원래 id 유지)
submission = pd.DataFrame({"id": test_ids, "Age": np.round(test_preds, 3)})
submission.to_csv("download/sample_submission.csv", index=False)
print("sample_submission.csv 파일 생성 완료!")

sample_submission.csv 파일 생성 완료!


---
코드 뜯어보기
---

- `VotingRegressor` : 여러 개의 개별 **회귀모델**을 조합하여 최종 예측값을 생성하는 앙상블 방법 중 하나, 각 모델의 예측값을 평균냄.

```python
# 개별 모델들 정의
```
- `LinearRegression()` : 선형회귀, 데이터의 선형적 관계 학습
    - 데이터가 정규화되지 않으면 성능이 저하될 수 있음. -> 스케일링 필요
- `RandomForestRegressor(n_estimators=100, random_state=42)` : 랜덤포레스트회귀, 여러 결정트리를 조합해 예측력 높임
    - `max_depth` : 트리의 깊이 제한 - 과적합 방지
    - `min_sample_split` : 최소한 몇 개의 샘플이 있어야 노드를 분할할지 설정
    - `min_sample_leaf` : 리프 노트에 필요한 최소 샘플 개수 지정.

- `GradientBoostingRegression(n_estimators=100, random_state=42)` : 여러 개의 약한 모델(결정트리)를 순차적으로 학습해 예측력 향상
    - `learning_rate` : 학습 속도 줄이면 모델이 점진적으로 개선 -> 과적합 방지 가능
    - `n_estimators` : 너무 크면 과적합될 수 있음 (적절한 값 찾기)
    - `subsample` : 1보다 작은 값 설정 -> 일부 데이터만 샘플링하여 훈련하면 과적합 방지